In [28]:
import numpy as np
import matplotlib.pyplot as plt
import rasterio as rio
from sklearn import svm
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import numpy as np
from pyrsgis import raster
from pyrsgis import ml
import random
from copy import copy
from itertools import cycle
from osgeo import gdal

In [29]:
featurefile = r'E:/Sentinel2/Sistan_Sentinel.tif'

In [30]:
lablefile = r'E:/Sentinel2/Sistan_Sentinel_Training.tif'

In [31]:
dsfeature , arrayfeature = raster.read(featurefile, bands = 'all')
arrayfeature = arrayfeature.astype(float)

In [32]:
print(arrayfeature)

[[[2112. 2070. 2056. ... 2527. 2510. 2522.]
  [2117. 2102. 2065. ... 2532. 2585. 2622.]
  [2125. 2125. 2116. ... 2545. 2543. 2592.]
  ...
  [1397. 1411. 1417. ... 2112. 2095. 2083.]
  [1454. 1442. 1470. ... 2085. 2096. 2086.]
  [1574. 1491. 1475. ... 2088. 2097. 2104.]]

 [[2308. 2288. 2270. ... 2888. 2868. 2850.]
  [2345. 2294. 2290. ... 2887. 2938. 2968.]
  [2352. 2349. 2333. ... 2915. 2913. 2962.]
  ...
  [1381. 1387. 1412. ... 2364. 2349. 2335.]
  [1437. 1456. 1471. ... 2357. 2343. 2342.]
  [1582. 1499. 1500. ... 2351. 2356. 2338.]]

 [[2745. 2711. 2721. ... 3568. 3536. 3547.]
  [2752. 2719. 2746. ... 3549. 3563. 3636.]
  [2796. 2756. 2762. ... 3549. 3587. 3665.]
  ...
  [1482. 1521. 1555. ... 2882. 2837. 2836.]
  [1582. 1607. 1599. ... 2869. 2869. 2847.]
  [1805. 1649. 1637. ... 2858. 2853. 2830.]]

 ...

 [[3002. 3002. 2976. ... 3866. 3931. 3931.]
  [3002. 3002. 2976. ... 3866. 3931. 3931.]
  [2941. 2941. 2920. ... 3919. 3909. 3909.]
  ...
  [1723. 1723. 1680. ... 3024. 3012. 301

In [33]:
arrayfeature.shape

(10, 866, 770)

In [34]:
for i in range(arrayfeature.shape[0]):
 minb = arrayfeature [i][:][:].min()
 maxb = arrayfeature [i][:][:].max()
 rangeb = maxb - minb
 for j in range(arrayfeature.shape[1]):
    for k in range(arrayfeature.shape[2]):
        arrayfeature[i][j][k] = (arrayfeature[i][j][k] - minb)/rangeb

In [35]:
print(arrayfeature)

[[[0.46893975 0.44932275 0.44278375 ... 0.6627744  0.65483419 0.66043905]
  [0.47127511 0.46426903 0.44698739 ... 0.66510976 0.68986455 0.70714619]
  [0.47501168 0.47501168 0.47080803 ... 0.67118169 0.67024755 0.69313405]
  ...
  [0.13498365 0.14152265 0.14432508 ... 0.46893975 0.46099953 0.45539468]
  [0.16160673 0.15600187 0.16907987 ... 0.45632882 0.4614666  0.45679589]
  [0.2176553  0.17888837 0.17141523 ... 0.45773003 0.46193368 0.46520318]]

 [[0.48655617 0.47918969 0.47255985 ... 0.70018416 0.69281768 0.68618785]
  [0.50018416 0.48139963 0.47992634 ... 0.69981584 0.71860037 0.72965009]
  [0.50276243 0.50165746 0.49576427 ... 0.71012891 0.70939227 0.72744015]
  ...
  [0.14511971 0.14732965 0.15653775 ... 0.50718232 0.50165746 0.49650092]
  [0.16574586 0.17274401 0.17826888 ... 0.50460405 0.49944751 0.49907919]
  [0.21915285 0.18858195 0.18895028 ... 0.50239411 0.50423573 0.49760589]]

 [[0.49627721 0.48654066 0.48940435 ... 0.73195876 0.72279496 0.72594502]
  [0.49828179 0.488831

In [36]:
print (arrayfeature.shape)

(10, 866, 770)


In [44]:
features = ml.array_to_chips(arrayfeature, y_size=7, x_size=7)

In [46]:
features.shape

(666820, 7, 7, 10)

In [47]:
dslable, arraylable = raster.read(lablefile)

In [50]:
np.unique(arraylable)

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9, 255], dtype=uint8)

In [51]:
arraylable[arraylable==255]=0

In [52]:
np.unique(arraylable)

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=uint8)

In [53]:
arraylable = arraylable.flatten()

In [54]:
arraylable.shape

(666820,)

In [57]:
features = features[arraylable != 0]
lable = arraylable[arraylable != 0]

In [58]:
np.unique(lable)

array([1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=uint8)